<미국 주식 용어>
https://inbongharu.tistory.com/entry/%EB%AF%B8%EA%B5%AD%EC%A3%BC%EC%8B%9D-%EB%B0%B0%EB%8B%B9%EC%A3%BC-%ED%88%AC%EC%9E%90-%EB%B0%B0%EB%8B%B9-%EA%B4%80%EB%A0%A8-%EC%9A%A9%EC%96%B4-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0

## 아래에 코드를 작성해주세요.

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import pandas as pd

df = pd.read_excel("/content/drive/Othercomputers/내 Mac/13차 추천시스템-2/sp500_fill_in_df_2.xlsx")
df = df.iloc[:, 1:]

In [66]:
df.isnull().sum()

shortName                  0
symbol                     0
country                    0
industry                   0
sector                     0
fullTimeEmployees          0
auditRisk                  0
boardRisk                  0
compensationRisk           0
shareHolderRightsRisk      0
overallRisk                0
dividendYield              0
payoutRatio                0
beta                       0
trailingPE                 0
averageVolume              0
marketCap                  0
enterpriseValue            0
profitMargins              0
heldPercentInsiders        0
heldPercentInstitutions    0
bookValue                  0
priceToBook                0
pegRatio                   0
enterpriseToEbitda         0
52WeekChange               0
recommendationMean         0
recommendationKey          0
numberOfAnalystOpinions    0
totalCash                  0
totalDebt                  0
quickRatio                 0
currentRatio               0
totalRevenue               0
returnOnAssets

In [140]:
df.head()

,shortName,symbol,country,industry,sector,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,dividendYield,payoutRatio,beta,trailingPE,averageVolume,marketCap,enterpriseValue,profitMargins,heldPercentInsiders,heldPercentInstitutions,bookValue,priceToBook,pegRatio,enterpriseToEbitda,52WeekChange,recommendationMean,recommendationKey,numberOfAnalystOpinions,totalCash,totalDebt,quickRatio,currentRatio,totalRevenue,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins
0,"Agilent Technologies, Inc.",A,United States,Diagnostics & Research,Healthcare,18400,10,4,5,9,8,0.0075,0.1916,1.029030,27.110132,2059173,36354879488,37087252480,0.19190,0.00286,0.90436,19.579,6.286327,1.86,18.012,-3.231388,2.1,buy,16,1175000064,2891000064,1.546,2.370,7040000000,0.10401,0.068,0.54588,0.29247,0.25114
1,"American Airlines Group, Inc.",AAL,United States,Airlines,Industrials,130800,7,4,6,9,7,0.0000,0.0000,1.567622,6.813187,23577585,12477214720,42808471552,0.03391,0.01004,0.55352,-8.840,0.000000,0.05,6.647,32.102276,2.9,hold,13,12316000256,42929000448,0.582,0.718,52260999168,0.03864,0.370,0.26094,0.12323,0.07937
2,Advance Auto Parts Inc.,AAP,United States,Specialty Retail,Consumer Cyclical,40000,3,5,6,2,2,0.0142,0.6686,1.096311,10.763744,2576167,4306116096,8581074432,0.03614,0.00949,0.97157,44.643,1.622651,4.20,9.214,-61.839546,3.0,hold,18,226499008,4636292096,0.202,1.244,11198105600,0.03311,0.013,0.44015,0.08317,0.05720
3,Apple Inc.,AAPL,United States,Consumer Electronics,Technology,164000,4,1,5,1,1,0.0050,0.1559,1.292349,32.956080,57155932,3068669460480,3100864413696,0.24493,0.00071,0.61232,3.953,49.354920,4.07,25.050,25.587380,2.0,buy,39,55872000000,109614997504,0.764,0.940,385095008256,0.20559,-0.025,0.43181,0.32145,0.29163
4,AbbVie Inc.,ABBV,United States,Drug Manufacturers—General,Healthcare,50000,9,7,7,9,9,0.0433,1.3435,0.536202,32.859188,5938331,246060302336,296734687232,0.13368,0.00130,0.71465,7.525,18.296345,-2.95,10.051,-6.815565,2.4,buy,25,6721999872,62461001728,0.659,0.960,56740999168,0.09443,-0.097,0.70959,0.52031,0.36979


In [141]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 수치형
num_features = ['marketCap', 'dividendYield', 'payoutRatio', 'beta', 'trailingPE', 'averageVolume', 'profitMargins',
                'heldPercentInsiders', 'heldPercentInstitutions', 'bookValue', 'priceToBook', 'pegRatio',
                'enterpriseToEbitda', '52WeekChange', 'recommendationMean', 'numberOfAnalystOpinions', 'totalCash',
                'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'returnOnAssets', 'revenueGrowth',
                'grossMargins', 'ebitdaMargins', 'operatingMargins']
# 범주형
cat_features = ['industry', 'sector']

# transformer 생성
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')

# preprocessor 생성
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)])

# df_preprocessed에 결과 저장
df_preprocessed = preprocessor.fit_transform(df)

df_preprocessed.shape

(490, 149)

In [142]:
df_preprocessed

<490x149 sparse matrix of type '<class 'numpy.float64'>'
	with 13720 stored elements in Compressed Sparse Row format>

In [71]:
len(df['sector'].unique())

11

In [78]:
df['sector'].value_counts()

Technology                75
Industrials               68
Healthcare                66
Financial Services        60
Consumer Cyclical         58
Consumer Defensive        37
Real Estate               31
Utilities                 30
Energy                    23
Basic Materials           21
Communication Services    21
Name: sector, dtype: int64

## MY

In [88]:
df[df['symbol'] == 'KO']['sector'].values[0]

'Consumer Defensive'

In [98]:
df[df['sector']== 'Consumer Defensive'].head()

,shortName,symbol,country,industry,sector,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,dividendYield,payoutRatio,beta,trailingPE,averageVolume,marketCap,enterpriseValue,profitMargins,heldPercentInsiders,heldPercentInstitutions,bookValue,priceToBook,pegRatio,enterpriseToEbitda,52WeekChange,recommendationMean,recommendationKey,numberOfAnalystOpinions,totalCash,totalDebt,quickRatio,currentRatio,totalRevenue,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins
11,Archer-Daniels-Midland Company,ADM,United States,Farm Products,Consumer Defensive,40360,6,5,2,5,4,0.0226,0.2070,0.809767,10.163112,2869203,44775260160,54319144960,0.04370,0.00672,0.83407,45.615,1.775732,-19.27,10.065,9.429884,2.2,buy,14,899000000,11550000128,0.335,1.510,101977997312,0.04414,0.018,0.07603,0.05292,0.04282
61,Brown Forman Inc,BF-B,United States,Beverages—Wineries & Distilleries,Consumer Defensive,5600,5,10,9,10,10,0.0122,0.4834,0.698087,42.621120,1404203,33429743616,34588024832,0.18519,0.01748,0.50406,6.829,10.048324,3.67,26.343,-3.610057,3.2,hold,16,374000000,2998000128,1.134,3.506,4228000000,0.10892,0.050,0.58988,0.31055,0.29163
62,Bunge Limited Bunge Limited,BG,United States,Farm Products,Consumer Defensive,23000,8,1,2,3,2,0.0257,0.2358,0.651297,10.331014,1283706,15849117696,19554144256,0.02330,0.00668,0.87820,66.109,1.572101,-1.06,6.644,13.473082,1.9,buy,10,3148999936,6617999872,0.608,1.829,66680000512,0.05675,-0.035,0.05841,0.04414,0.03802
76,"ConAgra Brands, Inc.",CAG,United States,Packaged Foods,Consumer Defensive,18600,9,8,7,7,8,0.0435,0.9296,0.533811,22.936620,4271808,15537844224,24843710464,0.05568,0.00464,0.86197,18.315,1.778324,1.53,11.394,-1.838458,2.7,hold,15,93900000,9467599872,0.239,0.762,12277000192,0.05296,0.022,0.26602,0.17761,0.15352
91,"Church & Dwight Company, Inc.",CHD,United States,Household & Personal Products,Consumer Defensive,5250,2,7,7,5,4,0.0111,0.6347,0.478825,58.844310,1573811,24003821568,26419800064,0.07492,0.00183,0.85491,15.020,6.542609,4.06,21.723,5.451226,2.9,hold,17,202800000,2589499904,0.561,1.185,5508199936,0.07867,0.102,0.42110,0.22080,0.18518


In [155]:
from sklearn.neighbors import NearestNeighbors

# k-NN 모델 훈련
def knn(stock_name):
  sector = df[df['symbol'] == stock_name]['sector'].values[0]
  df_sector = df[df['sector']== sector].reset_index()[1:]
  df_preprocessed = preprocessor.fit_transform(df_sector)
  stock_index = df_sector[df_sector['symbol'] == stock_name].index[0]

  df_preprocessed[stock_index].reshape(1, -1)

  knn = NearestNeighbors(n_neighbors = 6, metric='cosine')
  knn.fit(df_preprocessed)

  neighbor_indices = knn.kneighbors(df_preprocessed[stock_index].reshape(1, -1), return_distance=False)
  return df_sector.iloc[neighbor_indices[0]].drop('sector',axis=1)[1:]

In [100]:
#stock_index = df[df['symbol'] == 'KO'].index[0]
#df_preprocessed[stock_index].reshape(1, -1)
#neighbor_indices = knn.kneighbors(df_preprocessed[stock_index].reshape(1, -1), return_distance=True)
#print(neighbor_indices[1][0])
#print(df[df['symbol'] == 'A']['sector'])

In [101]:
#df.iloc[neighbor_indices[1][0]]

In [156]:
knn('KO')

,index,shortName,symbol,country,industry,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,dividendYield,payoutRatio,beta,trailingPE,averageVolume,marketCap,enterpriseValue,profitMargins,heldPercentInsiders,heldPercentInstitutions,bookValue,priceToBook,pegRatio,enterpriseToEbitda,52WeekChange,recommendationMean,recommendationKey,numberOfAnalystOpinions,totalCash,totalDebt,quickRatio,currentRatio,totalRevenue,returnOnAssets,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins
34,426,Target Corporation,TGT,United States,Discount Stores,440000,4,4,2,1,1,0.0332,0.7053,1.007855,22.722600,5543695,61625880576,77589364736,0.02490,0.00204,0.81697,25.143,5.277811,-2.12,11.683,-16.362030,2.3,buy,31,1320999936,18902999040,0.074,0.883,109271998464,0.04798,0.006,0.24817,0.06077,0.03618
2,62,Bunge Limited Bunge Limited,BG,United States,Farm Products,23000,8,1,2,3,2,0.0257,0.2358,0.651297,10.331014,1283706,15849117696,19554144256,0.02330,0.00668,0.87820,66.109,1.572101,-1.06,6.644,13.473082,1.9,buy,10,3148999936,6617999872,0.608,1.829,66680000512,0.05675,-0.035,0.05841,0.04414,0.03802
3,76,"ConAgra Brands, Inc.",CAG,United States,Packaged Foods,18600,9,8,7,7,8,0.0435,0.9296,0.533811,22.936620,4271808,15537844224,24843710464,0.05568,0.00464,0.86197,18.315,1.778324,1.53,11.394,-1.838458,2.7,hold,15,93900000,9467599872,0.239,0.762,12277000192,0.05296,0.022,0.26602,0.17761,0.15352
9,131,Dollar General Corporation,DG,United States,Discount Stores,170000,9,2,7,7,6,0.0145,0.2111,0.283811,15.669181,3043759,36465438720,53464989696,0.06186,0.00487,0.95096,27.060,6.143755,3.11,13.106,-32.675953,2.3,buy,26,313064000,17990352896,0.060,1.320,38436343808,0.07323,0.068,0.31317,0.10613,0.08651
35,437,"Tyson Foods, Inc.",TSN,United States,Farm Products,142000,10,10,5,10,10,0.0367,0.4498,0.771516,12.504786,3774978,18587787264,27148107776,0.02810,0.02302,0.84862,54.551,0.958186,4.35,7.673,-36.426660,2.7,hold,9,550000000,8929999872,0.558,1.663,53624999936,0.04010,0.001,0.08295,0.06598,0.04349


# 설명

## 'Sector 별로 주식을 추천해보자!'
<h2>

총 11개의 sector가 존재한다 <br/>
사용자가 원하는 분야의 데이터를 추출하여 KNN을 진행 TOP 5를 추천하는 시스템을 구축하고자 한다. <br/>

👉🏻 knn이라는 함수를 만들어 사용하고자 한다.<br/>
👉🏻 knn 함수는 입력받은 Ticker의 sector를 추출해 df에서 동일한 sector를 가진 데이터를 뽑아낸다. 이후 이 데이터를 활용하여 KNN을 이용해 가장 거리가 가까운 데이터 5개를 뽑아낸다.

</h2>
